In [13]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

device

device(type='cuda')

In [2]:
#!pip uninstall gymnasium
# !pip install gym
#!pip install shimmy

In [4]:
#!pip install pylint gymnasium stable_baselines3

In [12]:
import gymnasium as gym
from gymnasium import spaces
import numpy as np
import pandas as pd
import tempfile
import subprocess
import shutil
from transformers import pipeline
import os
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.vec_env import DummyVecEnv
import logging
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.logger import configure
import matplotlib.pyplot as plt

# Configurando ambiente

In [14]:
# Configurar logging
logging.basicConfig(filename='agentes_conversa.log', level=logging.INFO, format='%(asctime)s - %(message)s')

class DataAnalysisEnv(gym.Env):
    """
    Ambiente personalizado para agentes cooperativos de análise de dados.
    """

    def __init__(self, data_path):
        super(DataAnalysisEnv, self).__init__()

        # Carrega o dataset Walmart
        self.data = pd.read_csv(data_path)

        # Definição dos espaços de ação para os dois agentes (combinados em um Dict)
        self.action_space = spaces.MultiDiscrete([4, 5])  # Codificador tem 4 ações, Revisor tem 5 ações

        # Definindo um espaço para as observações numéricas
        self.max_codigo_length = 1000
        self.max_relatorio_length = 1500

        total_length = self.max_codigo_length + self.max_relatorio_length + 1  # Código, Relatório e o score

        # Espaço de observação agora é um único Box contínuo
        self.observation_space = spaces.Box(
            low=0,
            high=1,  # Normalizado para ficar entre 0 e 1
            shape=(total_length,),
            dtype=np.float32
        )

        # Inicialização das variáveis de estado
        self.state = {
            "codigo": np.zeros(self.max_codigo_length, dtype=np.int32),
            "relatorio": np.zeros(self.max_relatorio_length, dtype=np.int32),
            "score_relatorio": np.array([0.0], dtype=np.float32)
        }

        self.threshold_score = 135  # 90% de 150 pontos

        self.temp_dir = tempfile.mkdtemp()
        self.linter = 'pylint'

        # Modelos LLM para Codificador e Revisor
        self.codificador_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=0 if torch.cuda.is_available() else -1)
        self.revisor_llm = pipeline('text-generation', model='Salesforce/codegen-350M-mono', device=0 if torch.cuda.is_available() else -1)

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        if seed is not None:
            np.random.seed(seed)

        # Redefine o estado
        self.state = {
            "codigo": np.zeros(self.max_codigo_length, dtype=np.int32),
            "relatorio": np.zeros(self.max_relatorio_length, dtype=np.int32),
            "score_relatorio": np.array([0.0], dtype=np.float32)
        }

        # Retorna a observação como um vetor contínuo
        observation = np.concatenate([
            self.state['codigo'] / 65535,  # Normalizando valores Unicode entre 0 e 1
            self.state['relatorio'] / 65535,
            self.state['score_relatorio'] / 150
        ]).astype(np.float32)

        return observation, {}

    def step(self, actions):
        done = False
        info = {}

        # Ações dos agentes
        action_codificador = actions[0]
        action_revisor = actions[1]

        # Codificador gera código
        codigo_gerado = self.execute_action_codificador(action_codificador)
        self.state['codigo'][:len(codigo_gerado)] = [ord(c) for c in codigo_gerado]  # Usa ord() para valores Unicode

        # Log e print da ação do codificador
        logging.info(f"Codificador ação: {action_codificador} -> Código gerado:\n{codigo_gerado}")
        print(f"Codificador ação: {action_codificador} -> Código gerado:\n{codigo_gerado}")

        # Executa e avalia o código
        exec_result, exec_error = self.avaliar_codigo(codigo_gerado)
        linter_output = self.rodar_linter(codigo_gerado)

        codigo_esta_correto = exec_error is None

        # Revisor gera feedback
        feedback = self.execute_action_revisor(action_revisor, exec_error, linter_output)
        self.state['relatorio'][:len(feedback)] = [ord(c) for c in feedback]  # Usa ord() para valores Unicode

        # Log e print da ação do revisor
        logging.info(f"Revisor ação: {action_revisor} -> Feedback gerado:\n{feedback}")
        print(f"Revisor ação: {action_revisor} -> Feedback gerado:\n{feedback}")

        # Calcula a pontuação do relatório com critérios mais complexos
        self.state['score_relatorio'] = self.calcular_score_relatorio(feedback)
        normalized_score = min(self.state['score_relatorio'][0] / 150, 1.0)
        self.state['score_relatorio'][0] = normalized_score

        # Calcular recompensa detalhada
        reward_codificador = self.calcular_recompensa_codificador(exec_error, linter_output)
        reward_revisor = self.calcular_recompensa_revisor(feedback)
        combined_reward = (reward_codificador + reward_revisor) / 2  # Média das recompensas

        # Log e print das recompensas
        logging.info(f"Recompensa Codificador: {reward_codificador}, Recompensa Revisor: {reward_revisor}, Recompensa Combinada: {combined_reward}")
        print(f"Recompensa Codificador: {reward_codificador}, Recompensa Revisor: {reward_revisor}, Recompensa Combinada: {combined_reward}")

        # Verificar se o episódio terminou
        if self.state['score_relatorio'] >= self.threshold_score:
            done = True

        # Concatenar código, relatório e score em uma única observação
        observation = np.concatenate([
            self.state['codigo'] / 65535,  # Normalizando valores Unicode entre 0 e 1
            self.state['relatorio'] / 65535,
            self.state['score_relatorio'] / 150  # Normalizando o score entre 0 e 1
        ]).astype(np.float32)

        return observation, combined_reward, done, False, info  # Retornando o valor `False` para truncated

    def execute_action_codificador(self, action):
        actions_map = {
            0: "Processar os dados.",
            1: "Analisar os dados.",
            2: "Visualizar os resultados.",
            3: "Interpretar a análise."
        }

        action_text = actions_map.get(action, "")
        prompt = f"Escreva uma função Python para {action_text} usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal."

        # Gera o código, mas limita o número de tokens gerados
        generated = self.codificador_llm(prompt, max_new_tokens=50, num_return_sequences=1)
        codigo = generated[0]['generated_text']

        # Truncar o código gerado para o tamanho máximo permitido
        codigo = codigo[:1024]  # Truncando para 1024 caracteres

        return codigo

    def execute_action_revisor(self, action, exec_error, linter_output):
        actions_map = {
            0: "Análise estática do código.",
            1: "Executar o código.",
            2: "Propor refatorações.",
            3: "Aprovar o código.",
            4: "Melhorar o relatório."
        }

        action_text = actions_map.get(action, "")
        prompt = f"{action_text}\nCódigo:\n{''.join([chr(c) for c in self.state['codigo'] if c > 0])}\nErro:\n{exec_error}\nLinter:\n{linter_output}\n"

        # Usando o modelo BLOOM para gerar o feedback
        generated = self.revisor_llm(prompt, max_new_tokens=150, num_return_sequences=1)
        feedback = generated[0]['generated_text']

        # Truncar o feedback para o tamanho máximo permitido
        feedback = feedback[:1024]  # Truncando para 1024 caracteres

        return feedback

    def avaliar_codigo(self, codigo):
        try:
            temp_code_path = os.path.join(self.temp_dir, 'temp_code.py')
            with open(temp_code_path, 'w') as f:
                f.write(codigo)

            result = subprocess.run(['python', temp_code_path], capture_output=True, text=True, timeout=10)

            if result.returncode == 0:
                return result.stdout, None
            else:
                return None, result.stderr

        except Exception as e:
            return None, str(e)

    def rodar_linter(self, codigo):
        try:
            temp_code_path = os.path.join(self.temp_dir, 'temp_code.py')
            with open(temp_code_path, 'w') as f:
                f.write(codigo)

            result = subprocess.run([self.linter, temp_code_path], capture_output=True, text=True, timeout=10)

            return result.stdout if result.returncode == 0 else result.stderr

        except Exception as e:
            return str(e)

    def calcular_score_relatorio(self, relatorio):
        """
        Implementa o cálculo de pontuação do relatório com base em múltiplos critérios.
        """
        # Critérios para pontuação
        criterio_completude = min(len(relatorio) / 1500, 1.0) * 50
        criterio_coerencia = 50 if "refatoração" in relatorio or "refatorar" in relatorio else 0
        criterio_correcoes = 50 if "erro" in relatorio or "corrigir" in relatorio else 0

        return np.array([criterio_completude + criterio_coerencia + criterio_correcoes], dtype=np.float32)

    def calcular_recompensa_codificador(self, exec_error, linter_output):
        """
        Recompensa o codificador com base em erros de execução e resultados do linter.
        """
        reward = 0

        # Penalidade por erro de execução
        if exec_error:
            reward -= 10

        # Penalidade por problemas no linter
        if 'error' in linter_output.lower():
            reward -= 5

        # Bônus por sucesso
        if not exec_error and 'error' not in linter_output.lower():
            reward += 20

        return reward

    def calcular_recompensa_revisor(self, relatorio):
        """
        Recompensa o revisor com base na qualidade do feedback e melhorias propostas.
        """
        reward = 0

        # Bônus por incluir refatorações e correções
        if "refatorar" in relatorio or "melhorar" in relatorio:
            reward += 10

        if "erro" in relatorio or "corrigir" in relatorio:
            reward += 10

        return reward

    def render(self, mode="human"):
        print("Estado Atual:")
        print("Código:", ''.join([chr(c) for c in self.state['codigo'] if c > 0]))
        print("Relatório:", ''.join([chr(c) for c in self.state['relatorio'] if c > 0]))
        print("Score:", self.state['score_relatorio'])

    def close(self):
        shutil.rmtree(self.temp_dir)


# Checando o modelo

In [9]:
# Configuração do Logger
tmp_path = "./logs/"
new_logger = configure(tmp_path, ["stdout", "csv", "tensorboard"])

# Checagem de ambiente
env = DataAnalysisEnv('Walmart.csv')
check_env(env)
env = DummyVecEnv([lambda: env])

Logging to ./logs/


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Codificador ação: 3 -> Código gerado:
Escreva uma função Python para Interpretar a análise. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. Dados de vendedor do Amazon e Amazon de vendas por cada vendedor do website e datas para analise em realizar o análise temporal"""
"""A função analisa_analdata_


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 1 -> Feedback gerado:
Executar o código.
Código:
Escreva uma função Python para Interpretar a análise. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. Dados de vendedor do Amazon e Amazon de vendas por cada vendedor do website e datas para analise em realizar o análise temporal"""
"""A função analisa_analdata_
Erro:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 1
    Escreva uma função Python para Interpretar a análise. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal. Dados de vendedor do Amazon e Amazon de vendas por cada vendedor do website e datas para analise em realizar o análise temporal"""
            ^^^
SyntaxError: invalid syntax

Linter:

       1
       3
       4
       5






'''

import pandas as pd

def codify_cabecalho(dataframe, name):
    dataframe.insert(0, name, dataframe.pop(name))
    return dataframe


def nor

...

Revisor ação: 4 -> Feedback gerado:
Melhorar o relatório.
Código:
Escreva uma função Python para Visualizar os resultados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.



# In[8]:


from datetime import date
#%matplotlib inline 
from datetime import timedelta


resultados = pd.read_csv(r"Datasets/Walmarttitest import multipletests
import numpy as np

Erro:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 13
    resultados = pd.read_csv(r"Datasets/Walmart
                             ^
SyntaxError: unterminated string literal (detected at line 13)

Linter:

  File "/tmp/tmp8jm_m_ij/temp_code.py", line 15
    x = 0  # fortran comment
  comma keyword, not recognized ---> end
    from temp_script import write_x
Cython:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 1
    x = 1  # fortran comment
  line too short ---> comment
        else: continue













Recompensa Codificador: -10, Recompensa Revisor: 0, Recompensa 

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 2 -> Feedback gerado:
Propor refatorações.
Código:
Escreva uma função Python para Interpretar a análise. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.
'''

# Importar as bibliotecas necessárias
import requests
import pandas as pd
import numpy as np

# Função para análise sem física temporaltitest import multipletests
import numpy as np

Erro:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 2
    '''
    ^
SyntaxError: unterminated triple-quoted string literal (detected at line 9)

Linter:

  File "/tmp/tmp8jm_m_ij/temp_code.py", line 32
   ...
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 21
   ...

Linter:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 9
    """Anova ela base de Dados"""

def Anova_data():
  # Importar o dataset e pegar o dataset

  # Faz a anotação projeto 3x de análise temporaria
  url = "https://www.dropbox.com/s/q
Recompensa Codificador: -10, Recompensa Revisor: 0, Recompensa Combinad

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 1 -> Feedback gerado:
Executar o código.
Código:
Escreva uma função Python para Processar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.

'''

def processe_dataframe(df, coluna="Weekly_Sales", dataFrameColuna=False):
    '''
    Sobreescreva uma função que recebem física temporaltitest import multipletests
import numpy as np

Erro:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 1
    Escreva uma função Python para Processar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.
            ^^^
SyntaxError: invalid syntax

Linter:

  # Import the required Python module from numpy
#  >>> import numpy as np

#  Linter:

  # Define a function that takes an object of type numpy.ndarray as an argument, and returns the corresponding object of type float
   <<^
Functions
function (object)
  -> return object

'''
    if dataFrame

# Treinamento e teste do modelo

In [ ]:
# Modelo de treinamento
model = PPO("MlpPolicy", env, verbose=1,device = device)
model.set_logger(new_logger)

# Treinar o modelo
model.learn(total_timesteps=20000)

# Testar o modelo treinado
obs = env.reset()
for i in range(500):
    action, _states = model.predict(obs)
    obs, rewards, done, info = env.step(action)
    env.render()
    if done:
        obs = env.reset()

env.close()

# Para carregar o modelo mais tarde
# model = PPO.load("ppo_data_analysis")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Using cuda device
Codificador ação: 0 -> Código gerado:
Escreva uma função Python para Processar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.

import numpy as np
import pandas as pd
import os

#função para calcular os quantidades por milhão vendido
def calculoQVendas(dataFrame,coluna):


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 1 -> Feedback gerado:
Executar o código.
Código:
Escreva uma função Python para Processar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.

import numpy as np
import pandas as pd
import os

#função para calcular os quantidades por milhão vendido
def calculoQVendas(dataFrame,coluna):
Erro:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 1
    Escreva uma função Python para Processar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.
            ^^^
SyntaxError: invalid syntax

Linter:

  Traceback (most recent call last):
    File "/tmp/tmp8jm_m_ij/temp_code.py", line 25.3, in <module>
      dataFrame = pd.read_csv('/Users/A/Desktop/python/Crawling-Python/Walmart-sales.csv')
          ^
SyntaxError: invalid syntax:

Traceback (most recent call last):
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 1, in <module>
    dataFra

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 0 -> Feedback gerado:
Análise estática do código.
Código:
Escreva uma função Python para Analisar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.

#Função 1, por ser um código, se le o dataset de vendas da Walmart especificou, também como um módulo e, caso existisse, será import figure
from datetime import
Erro:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 1
    Escreva uma função Python para Analisar os dados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.
            ^^^
SyntaxError: invalid syntax

Linter:

	1) Nenhuma instrução foi aplicada. Não faz nada acessá-la.
	
	2) Variáveis repetidas devem ficar o mesmo
	
	3) Os elementos não deverão ser váctios: não é sempre um dos seguintes:
		- [str]
		- [False]
		- [float]
		- [int]
		- [tuple]
		- [bool]
		

#Testando análise devem conter um erro de código.
#  Todo(
Recomp

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Revisor ação: 4 -> Feedback gerado:
Melhorar o relatório.
Código:
Escreva uma função Python para Visualizar os resultados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.

Plote os gráficos seguras e assíncia de vendas, sem os intervalos de dados, durante o cenário.

Uma vez que criado, comimport figure
from datetime import
Erro:
  File "/tmp/tmp8jm_m_ij/temp_code.py", line 1
    Escreva uma função Python para Visualizar os resultados. usando a coluna de Date e Weekly_Sales do dataset Walmart.csv de vendas da Walmart para realizar uma análise temporal.
            ^^^
SyntaxError: invalid syntax

Linter:




"""

#Função

def most_popular(col):
    col.sort_values(by=col,  ascending=True, key=str.lower)
    col = col.loc[col['price'] >= col['threshold']]
    col = list(col[col['quantity']])

    return col  

def visualize(data, threshold=10, title='', figsize=(16,10)):
    """ Módulo de Visualizacao"""
    with plt

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


# Visualizando dados de métricas de treinamento

In [ ]:
import pandas as pd

# Carregar o log CSV gerado
log_data = pd.read_csv('/content/logs/progress.csv')

# Visualizar as primeiras linhas
print(log_data.head(20))

In [ ]:
from transformers import pipeline

generator = pipeline("text-generation", model="Salesforce/codegen-350M-mono")
prompt = "Write a Python function to calculate the factorial of a number"
result = generator(prompt, max_length=100)
print(result[0]['generated_text'])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Write a Python function to calculate the factorial of a number for integer inputs


def factorial(x):
    try:
        x=int(x)
        if (x>=1):
            result=1
            for i in range(1,x+1): #using while loop to find out the factorial of 1 to n
                result=result*i
            return (result) #returning the factorial

    except ValueError
